# CFGL Workflow

## Loading the required Modules

We load three modules respectively incorporating the functions for FGL, CFGL and the ccomputation of screening matrix. 

In [ ]:
import os
import numpy as np
import pandas as pd

os.chdir("/Users/seals/Documents/Github/RCFGL/Python_functions")
import ADMM_py_function_new as AP

os.chdir("/Users/seals/Documents/Github/RCFGL/Python_functions")
import CFGL_ADMM as CFGL_AP

os.chdir("/Users/seals/Documents/Github/RCFGL/Python_functions")
import get_screening as scr